In [11]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import cv2
import easygui
from rembg import remove
from PIL import Image
import joblib
import pandas as pd

In [2]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='10062024-0524.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [35]:
image_path = easygui.fileopenbox(title='Select Image File')
gas_path = easygui.fileopenbox(title='Select Gas File')

In [36]:
img = cv2.imread(str(image_path))
img = cv2.resize(img, (200, 200))
X_image = tf.keras.preprocessing.image.img_to_array(img)
X_image = np.expand_dims(X_image, axis=0)

In [37]:
def load_csv_data(file_path):
    csv_data = pd.read_csv(file_path).values.flatten()
    return np.array([csv_data])

def preprocess_data(data, scaler):
    data = scaler.transform(data)
    data = data.reshape((data.shape[0], data.shape[1], 1))
    return data
data = load_csv_data(gas_path)

scaler_path = 'scaler.pkl' 
scaler = joblib.load(scaler_path)
# Melakukan preprocessing pada data input
data = preprocess_data(data, scaler)

In [38]:
input_data_image = X_image
input_data_gas = data.astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data_image)
interpreter.set_tensor(input_details[1]['index'], input_data_gas)
interpreter.invoke()

In [39]:
predictions = interpreter.get_tensor(output_details[0]['index'])
# Melakukan prediksi
predictions = np.round(predictions[0],2).flatten()
print(f'Predictions: {predictions}')

Predictions: [0.]
